In [1]:
import numpy as np 
import torch
import random
import os

# customized 
import models.arch as models
from utils.model_zoo_server import ModelZooServer

In [2]:
model_names = [
    'vgg19',
    'vit_large_patch16_224',
    'efficientnet_b7', 
]

def model_zoo(model_names): 
    m_zoo = {}
    for model_name in model_names:
        model = models.network(model_name, pretrained=False, n_classes=10)
        
        ckpt_dir = 'experiments/mzoo/%s/model_best.pth.tar' % model_name
        if not os.path.exists(ckpt_dir):
            raise FileNotFoundError(f"Checkpoint not found at {ckpt_dir}")
        
        print(f'Loading checkpoint from {ckpt_dir}...')
        checkpoint = torch.load(ckpt_dir)
        
        model.load_state_dict(checkpoint['state_dict'])
        model.eval()
        model = model.cuda()

        m_zoo[model_name] = model
        """
        m_zoo[model_name] = model_name
        """

    return m_zoo

In [3]:
mzoo = model_zoo(model_names)

arch: vgg19, pretrained: False, n_classes: 10


/home/dsanyal7/miniconda3/envs/mia/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/dsanyal7/miniconda3/envs/mia/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Do not freeze layers for model: vgg19
Loading checkpoint from experiments/mzoo/vgg19/model_best.pth.tar...


/tmp/ipykernel_4137175/559065310.py:18: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(ckpt_dir)


arch: vit_large_patch16_224, pretrained: False, n_classes: 10
Freezing ViT-Large intermediate layers...
Loading checkpoint from experiments/mzoo/vit_large_patch16_224/model_best.pth.tar...
arch: efficientnet_b7, pretrained: False, n_classes: 10
Do not freeze layers for model: efficientnet_b7
Loading checkpoint from experiments/mzoo/efficientnet_b7/model_best.pth.tar...


In [4]:
psml_pareto_front = np.array([
    (72.18, 18.5 ),
    (84.62, 90.18),
    (74.1, 212.47),
])

In [26]:
eps = 100
sensitivity = 1

In [28]:
server

In [29]:
server.pareto_front

array([[ 72.18,  18.5 ],
       [ 84.62,  90.18],
       [ 74.1 , 212.47]])

In [30]:
server.model_zoo_map

{(72.18, 18.5): 'vgg19',
 (84.62, 90.18): 'vit_large_patch16_224',
 (74.1, 212.47): 'efficientnet_b7'}

In [64]:
#all_epsilons = [1,2,3,4,5,10,50,100,500,1000]
all_epsilons = [1,5,50,100,]

In [65]:
# queries = [(random.randint(50, 100), random.randint(10, 250)) for _ in range(20)]
# queries = [(72.18, 18.5) for _ in range(20)] # vgg19
queries = [(84.62, 90.18) for _ in range(4000)] # ViT

In [66]:
from collections import Counter

for eps in all_epsilons: 
    server = ModelZooServer(mzoo, model_names, psml_pareto_front, eps, sensitivity)
    
    print(f"\ncomputing goodput for eps={eps}...")
    utility = 0
    answered_queries = 0
    selected_points = [] 
    selected_models = [] 
    
    for query in queries:
        selected_element = server.l1_permute_and_flip_mechanism(query)
        # we can get none when the query is infeasible, then no point will have a high enough prob to be selected
        # print(query, ' -> ', selected_element)
        if selected_element is not None:
            answered_queries += 1
            selected_points.append(selected_element)
            selected_models.append(server.m_query(selected_element))
            
            utility += server.l1_score(float(selected_element[0]), float(selected_element[1]), query[0], query[1])
    
    print('utility score: ', utility / answered_queries)
    # for point, model in zip(selected_points, selected_models):
    #    print(point, ' -> ', model.__class__.__name__)
    
    model_counts = Counter([model.__class__.__name__ for model in selected_models])
    total_models = sum(model_counts.values())  # Total selections
    
    # Calculate and display proportions
    print("model selection:")
    for model_name, count in model_counts.items():
        proportion = count / total_models
        print(f"{model_name}: {count} ({proportion:.2%})")


computing goodput for eps=1...
utility score:  0.8136448125000204
model selection:
EfficientNet: 987 (24.68%)
VGG: 1447 (36.18%)
VisionTransformer: 1566 (39.15%)

computing goodput for eps=5...
utility score:  0.9286726375000235
model selection:
VisionTransformer: 2132 (53.30%)
VGG: 1587 (39.67%)
EfficientNet: 281 (7.03%)

computing goodput for eps=50...
utility score:  0.993624500000005
model selection:
VisionTransformer: 3590 (89.75%)
VGG: 410 (10.25%)

computing goodput for eps=100...
utility score:  0.9984761000000016
model selection:
VisionTransformer: 3902 (97.55%)
VGG: 98 (2.45%)
